## Byte-Encoding Representation

This is a subword tokenization of a vocabulary. An example of this from Sennrich et al. is as follows:

> vocab = ‘low’, ‘lowest’, ‘newer’, ‘wider’

> OOV = 'lower' --> 'low_', 'er_'

This is much more valuable than a UNK symbol. To build this representation, an iterative algorithm can be used to link together the most common segments, starting with character pairs. Below is the pseudo code provided by the original authors with a few changes.

In [5]:
import re
import sys
import collections

def get_stats(vocab):
    """For the given merge step, get the bigram counts, for every pair of symbols (strings).

    For each word and its freq in the corpus, add the number of instances for each of its subword parts.
    """
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram_pattern = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram_pattern + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

vocab = {
  'l o w </w>' : 5, 
  'f a r t h e s t </w>' : 5,
  'n e w e r </w>': 5,
  'w i d e r </w>': 5 }
num_merges = 5
transforms = set()
for i in range(num_merges):
    pairs = get_stats(vocab)

    try:
        best = max(pairs, key=pairs.get)
    except ValueError:
        break
    if pairs[best] < 2:
        print('no pair has frequency > 1. Stopping\n')
        break
    transforms.add(best)
    vocab = merge_vocab(best, vocab)
    print(best)

('e', 'r')
('er', '</w>')
('l', 'o')
('lo', 'w')
('low', '</w>')
